In [2]:
import pandas as pd
import os
import sys
import pickle
import datetime
import time
import re
from sqlalchemy import create_engine
from sqlalchemy import text
import sqlalchemy
import psycopg2

In [3]:
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GroupShuffleSplit
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline

2026-02-20 15:03:50.132100: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-20 15:03:50.165222: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-20 15:03:50.955968: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import joblib

In [6]:
TOKENIZERS_PARALLELISM=False

In [7]:
STOP_WORDS = { "the",
    "and", "or", "the", "a", "an", "to", "of", "in", "on", "for", "with",
    "by", "at", "from", "as", "is", "are", "this", "that", 'tv', 'collection',
    'episodes' ,'episode', 'и', 'ну', 'не', 'ссср','shorts', 'y', 'kidsmania',
    'un', 'el', 'de', 'be'
}

In [8]:
#
def tokenize_clean(text):
    tokens = re.findall(r"\b[^\W_]+\b", text.lower(), flags=re.UNICODE)
    return [ t for t in tokens if not t.isdigit() and t not in STOP_WORDS ]

In [138]:
# get reels from sql and learn model

In [17]:
conn_string = os.environ['PG_DB2']
db = create_engine(conn_string.replace('multdb', 'youtube'))
conn = db.connect()

In [18]:
req1 = '''select iyr.yt_reel_id, reel_name, yt_ch_name, cartoon, brand_id, product_id,language from intl_yt_reels iyr 
join intl_yt_channels2reels iyrcr 
on iyr.yt_reel_id = iyrcr.yt_reel_id 
join intl_yt_channels iyc 
on iyrcr.yt_ch_id = iyc.yt_ch_id 
where iyc."language" in ('США','Английский', 'Испанский', 'Арабский', 'Португальский','Китайский')
and brand_id like 'BR%' 
and date < '2026-02-01' '''

In [19]:
df_in = pd.read_sql(text(req1), conn)
conn.close()

In [20]:
df_in['name'] = df_in.yt_ch_name + ' '+df_in.reel_name
df_in["tokens"] = df_in["name"].apply(tokenize_clean)

In [21]:
texts = df_in['tokens'].str.join(' ').to_list() #df_in["tokens"].to_list()
labels = df_in['brand_id'].to_list()

In [ ]:
########################

In [27]:
X_train, X_test, y_train, y_test = train_test_split( texts, labels, test_size=0.25, random_state=54321, stratify= labels)

In [28]:
vectorizer = TfidfVectorizer(
    analyzer="word",
    ngram_range=(1, 2),     # униграммы + биграммы
    min_df=2,               # убрать шум
    max_df=0.95,
    max_features=50000,
    lowercase=True
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [29]:
def compute_smooth_weights(y, alpha=0.5):
    counter = Counter(y)
    N = len(y)
    weights = {cls: (N / count) ** alpha for cls, count in counter.items()}
    min_w = min(weights.values())
    weights = {k: v / min_w for k, v in weights.items()}
    return weights

class_weights_dict = compute_smooth_weights(y_train, alpha=0.5)

classes = sorted(np.unique(y_train))
class_weights = [class_weights_dict[c] for c in classes]

In [30]:
model = CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.05,
    loss_function="MultiClass",
    eval_metric="TotalF1",
    class_weights=class_weights,
    random_seed=42,
    verbose=100,
    early_stopping_rounds=100
)

model.fit(
    X_train_vec,
    y_train,
    eval_set=(X_test_vec, y_test),
    use_best_model=True
)

0:	learn: 0.4838729	test: 0.4853641	best: 0.4853641 (0)	total: 103ms	remaining: 1m 42s
100:	learn: 0.9660084	test: 0.9669797	best: 0.9669797 (97)	total: 5.04s	remaining: 44.9s
200:	learn: 0.9970619	test: 0.9952563	best: 0.9964880 (173)	total: 9.97s	remaining: 39.6s
300:	learn: 0.9997511	test: 0.9987652	best: 0.9987652 (257)	total: 14.9s	remaining: 34.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9987651889
bestIteration = 257

Shrink model to first 258 iterations.


In [38]:
X2 = vectorizer.transform(texts2)
y_pred = model.predict(X2)
print(classification_report(labels2, y_pred))
print(confusion_matrix(labels2, y_pred))

              precision    recall  f1-score   support

      BR0071       1.00      1.00      1.00        25
      BR0133       1.00      1.00      1.00         1
      BR0175       1.00      1.00      1.00         2
      BR0176       1.00      1.00      1.00        13
      BR0186       1.00      1.00      1.00         1
      BR0192       1.00      1.00      1.00         5
      BR0234       1.00      1.00      1.00         2
      BR0251       1.00      1.00      1.00         2

    accuracy                           1.00        51
   macro avg       1.00      1.00      1.00        51
weighted avg       1.00      1.00      1.00        51

[[25  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  5  0  0]
 [ 0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  2]]


In [1]:
######################## Analyze model - words importance

In [27]:
import shap
from catboost import Pool


In [77]:

#model = pipeline.named_steps["model"]   # если pipeline
#vectorizer = pipeline.named_steps["tfidf"]

feature_names = vectorizer.get_feature_names_out()

importances = model.get_feature_importance()

df_imp = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values("importance", ascending=False)

In [78]:
df_imp

,feature,importance
1837,rockoons,10.676273
368,claymotions,6.198612
1332,musipaches,5.111920
1031,kids,5.094102
1310,mug,4.253968
...,...,...
986,journal,0.000000
987,journal cartoon,0.000000
988,juega,0.000000
989,juega baila,0.000000


In [41]:
from catboost import Pool

In [42]:
X_train_vec = vectorizer.transform(X_train)

train_pool = Pool(X_train_vec, label=y_train)

# SHAP
shap_values = model.get_feature_importance(
    data=train_pool,
    type="ShapValues"
)

In [91]:
#######

In [142]:
y = y_train

In [143]:


class_mean_tfidf = {}

y = np.asarray(y)  # метка для каждого документа
classes = np.unique(y)

for cls in classes:
    mask = (y == cls)
    X_cls = X_tfidf[mask]
    
    # среднее по строкам (документам)
    mean_vector = X_cls.mean(axis=0)
    
    # перевод в 1D массив
    mean_vector = np.asarray(mean_vector).ravel()
    
    class_mean_tfidf[cls] = mean_vector

In [145]:
global_mean = np.asarray(X_tfidf.mean(axis=0)).ravel()

result = {}
top_n = 20

for cls, mean_vector in class_mean_tfidf.items():
    diff = mean_vector - global_mean
    
    top_indices = diff.argsort()[-top_n:][::-1]
    
    result[cls] = pd.DataFrame({
        "feature": feature_names[top_indices],
        "class_mean": mean_vector[top_indices],
        "diff_from_global": diff[top_indices]
    })

In [146]:
global_mean = np.asarray(X_tfidf.mean(axis=0)).ravel()

class_diff = {}

for cls, mean_vector in class_mean_tfidf.items():
    diff = mean_vector - global_mean
    
    top_indices = diff.argsort()[-top_n:][::-1]
    print(cls)
    class_diff[cls] = pd.DataFrame({
        "feature": feature_names[top_indices],
        "diff_from_global": diff[top_indices]
    })

BR0071
BR0087
BR0133
BR0158
BR0175
BR0176
BR0184
BR0186
BR0192
BR0234
BR0251
BR0284


In [165]:
result['BR0192']

,feature,class_mean,diff_from_global
0,peter,0.227922,0.223984
1,peter wolf,0.227922,0.223984
2,wolf,0.226188,0.222166
3,adventures peter,0.215354,0.211633
4,case,0.211112,0.207379
5,adventures,0.201967,0.197195
6,wolf 11minute,0.182791,0.179633
7,11minute,0.177098,0.173462
8,11minute cartoon,0.172791,0.169227
9,giggle mug,0.143389,0.132264


In [166]:
result['BR0184']

,feature,class_mean,diff_from_global
0,pirate,0.398865,0.397434
1,pirate school,0.387918,0.386943
2,school,0.387918,0.386943
3,school 11minute,0.303988,0.303224
4,11minute cartoon,0.229812,0.226249
5,11minute,0.228834,0.225199
6,giggle,0.208198,0.197073
7,giggle mug,0.208198,0.197073
8,mug,0.208198,0.197073
9,cartoon kids,0.159390,0.145443


In [161]:
######

In [162]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

class_vectors = np.vstack([
    class_mean_tfidf[cls] for cls in sorted(class_mean_tfidf)
])

sim_matrix = cosine_similarity(class_vectors)

pd.DataFrame(
    sim_matrix,
    index=sorted(class_mean_tfidf),
    columns=sorted(class_mean_tfidf)
)

,BR0071,BR0087,BR0133,BR0158,BR0175,BR0176,BR0184,BR0186,BR0192,BR0234,BR0251,BR0284
BR0071,1.000000,0.053027,0.108180,0.068054,0.038043,0.209201,0.049938,0.329232,0.099055,0.109977,0.185554,0.204369
BR0087,0.053027,1.000000,0.000000,0.020720,0.005816,0.018973,0.043806,0.047246,0.047697,0.020387,0.028186,0.012915
BR0133,0.108180,0.000000,1.000000,0.000000,0.000000,0.099988,0.000000,0.024460,0.000000,0.033522,0.081871,0.056827
BR0158,0.068054,0.020720,0.000000,1.000000,0.077741,0.020497,0.298063,0.055138,0.273753,0.210872,0.021212,0.022769
BR0175,0.038043,0.005816,0.000000,0.077741,1.000000,0.008629,0.054491,0.027865,0.056521,0.056511,0.070648,0.015410
BR0176,0.209201,0.018973,0.099988,0.020497,0.008629,1.000000,0.015108,0.137391,0.029759,0.062540,0.080834,0.117496
BR0184,0.049938,0.043806,0.000000,0.298063,0.054491,0.015108,1.000000,0.043131,0.311538,0.153947,0.014060,0.015992
BR0186,0.329232,0.047246,0.024460,0.055138,0.027865,0.137391,0.043131,1.000000,0.113011,0.122638,0.140294,0.309148
BR0192,0.099055,0.047697,0.000000,0.273753,0.056521,0.029759,0.311538,0.113011,1.000000,0.175113,0.023521,0.079519
BR0234,0.109977,0.020387,0.033522,0.210872,0.056511,0.062540,0.153947,0.122638,0.175113,1.000000,0.117504,0.092396


In [164]:
top_words = {
    cls: set(result[cls]["feature"])
    for cls in result
}

for c1 in top_words:
    for c2 in top_words:
        if c1 < c2:
            overlap = len(top_words[c1] & top_words[c2])
            print(c1, c2, overlap)

BR0071 BR0087 0
BR0071 BR0133 0
BR0071 BR0158 0
BR0071 BR0175 0
BR0071 BR0176 0
BR0071 BR0184 0
BR0071 BR0186 0
BR0071 BR0192 0
BR0071 BR0234 0
BR0071 BR0251 1
BR0071 BR0284 0
BR0087 BR0133 0
BR0087 BR0158 1
BR0087 BR0175 0
BR0087 BR0176 0
BR0087 BR0184 1
BR0087 BR0186 0
BR0087 BR0192 1
BR0087 BR0234 0
BR0087 BR0251 0
BR0087 BR0284 0
BR0133 BR0158 0
BR0133 BR0175 0
BR0133 BR0176 0
BR0133 BR0184 0
BR0133 BR0186 0
BR0133 BR0192 0
BR0133 BR0234 1
BR0133 BR0251 1
BR0133 BR0284 0
BR0158 BR0175 0
BR0158 BR0176 0
BR0158 BR0184 6
BR0158 BR0186 0
BR0158 BR0192 5
BR0158 BR0234 2
BR0158 BR0251 0
BR0158 BR0284 0
BR0175 BR0176 0
BR0175 BR0184 0
BR0175 BR0186 0
BR0175 BR0192 0
BR0175 BR0234 1
BR0175 BR0251 1
BR0175 BR0284 0
BR0176 BR0184 0
BR0176 BR0186 0
BR0176 BR0192 0
BR0176 BR0234 0
BR0176 BR0251 0
BR0176 BR0284 0
BR0184 BR0186 1
BR0184 BR0192 7
BR0184 BR0234 2
BR0184 BR0251 0
BR0184 BR0284 0
BR0186 BR0192 2
BR0186 BR0234 0
BR0186 BR0251 0
BR0186 BR0284 1
BR0192 BR0234 2
BR0192 BR0251 0
BR0192 B